<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

<p> <b>Описание задачи:</b>
<p>Создать базовый класс Invoice в C#, который будет представлять информацию о
фактурах за поставленные товары или оказанные услуги. На основе этого класса
разработать 2-3 производных класса, демонстрирующих принципы наследования и
полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и
методы, а также переопределены некоторые методы базового класса для
демонстрации полиморфизма.</p>
<p><b>Требования к базовому классу Invoice:</b>
<ul>
 <li><b>Атрибуты:</b> Номер фактуры (InvoiceNumber), Дата выдачи (IssueDate), Общая сумма (TotalAmount).</li>
 <li><b>Методы:</b>
    <ul>
        <li>CalculateTotal(): метод для расчета общей суммы по фактуре. </li>
        <li>AddLine(LineItem lineItem): метод для добавления позиции в фактуру.</li>
        <li>RemoveLine(LineItem lineItem): метод для удаления позиции из
фактуры.</li>
    </ul>
  </li>
  </ul>
  </p>

<p><b>Требования к производным классам: </b> </p>

<p>
<ol>
 <li>ТоварнаяФактура (GoodsInvoice): Должна содержать дополнительные
атрибуты, такие как Дата поставки (SupplyDate). Метод AddLine() должен
быть переопределен для добавления информации о дате поставки товара
при добавлении позиции. </li>
 <li>УслуговаяФактура (ServiceInvoice): Должна содержать дополнительные
атрибуты, такие как Дата оказания услуги (ServiceDate).
Метод RemoveLine() должен быть переопределен для добавления
информации о причине аннулирования услуги при удалении позиции. </li>
  <li>КомбинированнаяФактура (CombinedInvoice) (если требуется третий класс):
Должна содержать дополнительные атрибуты, такие как Наличие возврата
(ReturnAllowed). Метод CalculateTotal() должен быть переопределен для
учета возможного возврата товара или услуги при расчете общей суммы. </li> 
 </ol>
</p>

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [11]:
using System;
using System.Collections.Generic;
using System.Linq;

//Базовый класс для создания объекта фактуры
public class LineItem{
    public string Description { get; set; }
    public decimal UnitPrice { get; set; }
    public int Quantity { get; set; }

    public decimal Total => UnitPrice * Quantity;
    
    public LineItem(string description, decimal unitPrice, int quantity)
    {
        Description = description;
        UnitPrice = unitPrice;
        Quantity = quantity;
    }
}

//Класс для создания объекта товарной фактуры
public class GoodsLineItem : LineItem
{
    public DateTime SupplyDate { get; set; }


    public GoodsLineItem(string description, decimal unitPrice, int quantity, DateTime supplyDate)
        : base(description, unitPrice, quantity)
    {
        SupplyDate = supplyDate;
    }

}

//Класс для создания объекта сервисной фактуры
public class ServiceLineItem : LineItem
{
    public DateTime ServiceDate { get; set; }


    public ServiceLineItem(string description, decimal unitPrice, int quantity, DateTime serviceDate)
        : base(description, unitPrice, quantity)
    {
        ServiceDate = serviceDate;
    }

}

//Класс для создания объекта комбинированной фактуры
public class CombinedLineItem : LineItem
{
    public bool ReturnAllowed { get; set; }


    public CombinedLineItem(string description, decimal unitPrice, int quantity, bool returnAllowed)
        : base(description, unitPrice, quantity)
    {
        ReturnAllowed = returnAllowed;
    }

}

public delegate void InvoiceModifiedHandler(Invoice invoice, string action, LineItem item);
public delegate void TotalCalculatedHandler(Invoice invoice, decimal total);
public delegate bool LineItemFilter(LineItem item);

//Абстрактный класс фактур в целом
public abstract class Invoice{
    public string InvoiceNumber { get; protected set;}
    public DateTime IssueDate {get; protected set;}
    public decimal TotalAmount => CalculateTotal();
    public IReadOnlyList<LineItem> LineItems => _lineItems.AsReadOnly();


    private readonly Dictionary<string, LineItem> _lineItemsDict = new Dictionary<string, LineItem>();
    private readonly List<LineItem> _lineItems = new List<LineItem>();
    private readonly HashSet<string> _processedItems = new HashSet<string>();

    public event InvoiceModifiedHandler InvoiceModified;
    public event TotalCalculatedHandler TotalCalculated;


    protected Invoice(string invoiceNumber, DateTime issueDate)
    {
        InvoiceNumber = invoiceNumber;
        IssueDate = issueDate;
    }

    public virtual decimal CalculateTotal()
    {
        return _lineItems.Sum(item => item.Total);
    }

    public virtual void AddLine(LineItem lineItem)
    {
        if (lineItem == null)
            throw new ArgumentNullException(nameof(lineItem));

        // Использование коллекции Dictionary для быстрого поиска
        string key = $"{lineItem.Description}_{lineItem.UnitPrice}";
        if (_lineItemsDict.ContainsKey(key))
        {
            // Если товар уже есть, обновляем количество
            var existingItem = _lineItemsDict[key];
            existingItem.Quantity += lineItem.Quantity;
        }
        else
        {
            _lineItems.Add(lineItem);
            _lineItemsDict[key] = lineItem;
            _processedItems.Add(key);
        }

        // Вызов события при изменении фактуры
        InvoiceModified?.Invoke(this, "ADDED", lineItem);
    }

     public virtual void RemoveLine(LineItem lineItem)
    {
        if (lineItem == null)
            throw new ArgumentNullException(nameof(lineItem));

        if (_lineItems.Remove(lineItem))
        {
            string key = $"{lineItem.Description}_{lineItem.UnitPrice}";
            _lineItemsDict.Remove(key);
            _processedItems.Remove(key);

            // Вызов события при изменении фактуры
            InvoiceModified?.Invoke(this, "REMOVED", lineItem);
        }
    }

    // Метод с использованием делегата для фильтрации
    public IEnumerable<LineItem> FilterLineItems(LineItemFilter filter)
    {
        return _lineItems.Where(item => filter(item));
    }

    // Метод с использованием делегата Func
    public decimal CalculateFilteredTotal(Func<LineItem, bool> filter)
    {
        return _lineItems.Where(filter).Sum(item => item.Total);
    }

    // Метод с использованием Action
    public void ProcessLineItems(Action<LineItem> action)
    {
        _lineItems.ForEach(action);
    }

    protected virtual void OnInvoiceModified(string action, LineItem item)
    {
        InvoiceModified?.Invoke(this, action, item);
    }


    public abstract string GetInvoiceType();


}


//Класс товарной фактуры
class GoodsInvoice: Invoice{
    public DateTime SupplyDate {get; protected set;}

    public GoodsInvoice(string invoiceNumber, DateTime issueDate, DateTime supplyDate) 
            : base(invoiceNumber, issueDate)
    {
        SupplyDate = supplyDate;
    }

    public override void AddLine(LineItem lineItem)
    {
        if (lineItem is GoodsLineItem goodsLine)
        {
            base.AddLine(goodsLine);
        }
        else
        {
            throw new ArgumentException("Товарная фактура поддерживает только тип GoodsLineItem");
        }
    }

    public override string GetInvoiceType()
    {
        return "Товарная фактура";
    }
}


//Интерфейс сервисной фактуры с причиной отмены услуги
public interface IServiceInvoice
{
    DateTime ServiceDate { get; set; }
    void RemoveLine(LineItem lineItem, string cancellationReason);
}


//Класс сервисной фактуры
class ServiceInvoice: Invoice, IServiceInvoice{
    public DateTime ServiceDate {get; set;}
    
    // Событие специфичное для сервисной фактуры
    public event Action<string> ServiceCancelled;

    public ServiceInvoice(string invoiceNumber, DateTime issueDate, DateTime serviceDate) 
            : base(invoiceNumber, issueDate)
    {
        ServiceDate = serviceDate;
    }

    public override void RemoveLine(LineItem lineItem)
    {
        base.RemoveLine(lineItem);
        Console.WriteLine("Услуга аннулирована. Причина: Причина не указана");
        ServiceCancelled?.Invoke("Причина не указана");
    }

    void IServiceInvoice.RemoveLine(LineItem lineItem, string cancellationReason)
    {
        base.RemoveLine(lineItem);
        Console.WriteLine($"Услуга аннулирована. Причина: {cancellationReason}");
        ServiceCancelled?.Invoke(cancellationReason);

    }

    public override string GetInvoiceType()
    {
        return "Сервисная фактура";
    }
}

//Класс комбинированной фактуры
class CombinedInvoice: Invoice{
    public bool ReturnAllowed { get; set; }
    private decimal _returnAmount;

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed = false) 
            : base(invoiceNumber, issueDate)
    {
        ReturnAllowed = returnAllowed;
    }

    public void SetReturnAmount(decimal amount)
        {
            if (!ReturnAllowed)
                throw new InvalidOperationException("Возврат не разрешен для этой фактуры");
            
            _returnAmount = amount;

            // Вызов события при изменении суммы возврата
            OnInvoiceModified("RETURN_UPDATED", null);
        }

    public decimal CalculateReturn()
    {
        return ReturnAllowed ? _returnAmount : 0;
    }

    public override decimal CalculateTotal(){
        var total = base.CalculateTotal();
        return total - CalculateReturn();
    }

    public override string GetInvoiceType()
    {
        return "Комбинированная фактура";
    }
}

//Класс для создания фактур (паттерн фабрика)
public static class InvoiceFactory
    {
        public static Invoice CreateGoodsInvoice(string number, DateTime issueDate, DateTime supplyDate)
        {
            return new GoodsInvoice(number, issueDate, supplyDate);
        }

        public static Invoice CreateServiceInvoice(string number, DateTime issueDate, DateTime serviceDate)
        {
            return new ServiceInvoice(number, issueDate, serviceDate);
        }

        public static Invoice CreateCombinedInvoice(string number, DateTime issueDate, bool returnAllowed)
        {
            return new CombinedInvoice(number, issueDate, returnAllowed);
        }
    }

// Класс для управления коллекцией фактур
public class InvoiceManager
{
    private readonly List<Invoice> _invoices = new List<Invoice>();
    private readonly Dictionary<string, Invoice> _invoiceDictionary = new Dictionary<string, Invoice>();

    public event Action<Invoice> InvoiceAdded;
    public event Action<Invoice> InvoiceRemoved;

    public void AddInvoice(Invoice invoice)
    {
        _invoices.Add(invoice);
        _invoiceDictionary[invoice.InvoiceNumber] = invoice;
        InvoiceAdded?.Invoke(invoice);

        // Подписка на события фактуры
        invoice.InvoiceModified += OnInvoiceModified;
        invoice.TotalCalculated += OnTotalCalculated;
    }

    public bool RemoveInvoice(string invoiceNumber)
    {
        if (_invoiceDictionary.TryGetValue(invoiceNumber, out var invoice))
        {
            _invoices.Remove(invoice);
            _invoiceDictionary.Remove(invoiceNumber);
            InvoiceRemoved?.Invoke(invoice);
            return true;
        }
        return false;
    }

    public Invoice GetInvoice(string invoiceNumber)
    {
        _invoiceDictionary.TryGetValue(invoiceNumber, out var invoice);
        return invoice;
    }

    public IEnumerable<Invoice> GetInvoicesByType<T>() where T : Invoice
    {
        return _invoices.Where(i => i is T);
    }

    private void OnInvoiceModified(Invoice invoice, string action, LineItem item)
    {
        Console.WriteLine($"Фактура {invoice.InvoiceNumber} изменена: {action} {item?.Description ?? "N/A"}");
    }

    private void OnTotalCalculated(Invoice invoice, decimal total)
    {
        Console.WriteLine($"Рассчитана общая сумма для фактуры {invoice.InvoiceNumber}: {total:C}");
    }
}

// Демонстрация использования

var invoiceManager = new InvoiceManager();

// Подписка на события менеджера фактур
invoiceManager.InvoiceAdded += invoice => 
    Console.WriteLine($"Добавлена новая фактура: {invoice.InvoiceNumber}");
invoiceManager.InvoiceRemoved += invoice => 
    Console.WriteLine($"Удалена фактура: {invoice.InvoiceNumber}");

var goodsInvoice = InvoiceFactory.CreateGoodsInvoice(
    "INV-001", 
    DateTime.Now, 
    DateTime.Now.AddDays(7));
                
var serviceInvoice = InvoiceFactory.CreateServiceInvoice(
    "INV-002",
    DateTime.Now,
    DateTime.Now.AddDays(3));
                
var combinedInvoice = InvoiceFactory.CreateCombinedInvoice(
    "INV-003",
    DateTime.Now,
    true);
    

var product = new GoodsLineItem("Ноутбук", 1500, 2, DateTime.Now.AddDays(7));
var service = new ServiceLineItem("Техническое обслуживание", 300, 1, DateTime.Now.AddDays(3));
var product2 = new GoodsLineItem("Мышь", 50, 3, DateTime.Now.AddDays(7));

goodsInvoice.AddLine(product);
goodsInvoice.AddLine(product2); // Демонстрация работы Dictionary - будет обновлено количество
serviceInvoice.AddLine(service);

((CombinedInvoice)combinedInvoice).SetReturnAmount(200);
combinedInvoice.AddLine(new CombinedLineItem("Планшет", 800, 1, true));

// Использование делегатов для фильтрации
Console.WriteLine("\n--- Фильтрация товаров дороже 1000 ---");
var expensiveItems = goodsInvoice.FilterLineItems(item => item.UnitPrice > 1000);
foreach (var item in expensiveItems)
{
    Console.WriteLine($"Дорогой товар: {item.Description} - {item.UnitPrice:C}");
}

// Использование Func делегата
Console.WriteLine("\n--- Расчет общей суммы дорогих товаров ---");
decimal expensiveTotal = goodsInvoice.CalculateFilteredTotal(item => item.UnitPrice > 1000);
Console.WriteLine($"Общая сумма дорогих товаров: {expensiveTotal:C}");

// Использование Action делегата
Console.WriteLine("\n--- Обработка всех позиций ---");
goodsInvoice.ProcessLineItems(item => 
    Console.WriteLine($"Обработка: {item.Description} x {item.Quantity}"));

// Демонстрация работы с коллекциями
Console.WriteLine("\n--- Все товарные фактуры ---");
var goodsInvoices = invoiceManager.GetInvoicesByType<GoodsInvoice>();
foreach (var invoice in goodsInvoices)
{
    Console.WriteLine($"Товарная фактура: {invoice.InvoiceNumber}");
}

// Вывод общей информации
var invoices = new List<Invoice> { goodsInvoice, serviceInvoice, combinedInvoice };
foreach (var invoice in invoices)
{
    Console.WriteLine($"\n{invoice.GetInvoiceType()}:");
    Console.WriteLine($"Общая сумма: {invoice.CalculateTotal()}");
    Console.WriteLine($"Количество позиций: {invoice.LineItems.Count}");
}

// Демонстрация отмены сервиса
((IServiceInvoice)serviceInvoice).RemoveLine(service, "Клиент отказался от услуги");


--- Фильтрация товаров дороже 1000 ---
Дорогой товар: Ноутбук - ¤1,500.00

--- Расчет общей суммы дорогих товаров ---
Общая сумма дорогих товаров: ¤3,000.00

--- Обработка всех позиций ---
Обработка: Ноутбук x 2
Обработка: Мышь x 3

--- Все товарные фактуры ---

Товарная фактура:
Общая сумма: 3150
Количество позиций: 2

Сервисная фактура:
Общая сумма: 300
Количество позиций: 1

Комбинированная фактура:
Общая сумма: 600
Количество позиций: 1
Услуга аннулирована. Причина: Клиент отказался от услуги
